# Projet MAP569 Cédric JAVAULT (Master IA Year 1)

## 1. Préliminaires

In [1]:
import numpy as np
import csv
from datetime import datetime, date, time
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix,precision_score,recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


## 2. Lire le fichier et organiser les données

Le fichier de data comporte 5380 lignes (+ l'entete) et 19 colonnes. La première étape est de charger ces données et surtout de les organiser de manière pertinente pour la suite. Parmi ces 19 colonnes, il y a un Id_Customer qui n'a aucune raison d'être pertinent pour la suite. Je n'intégre évidemment pas le Y dans les données d'entrée. En revanche, je garde toutes les autres colonnes et ne fais **pas de feature selection** : il y a relativement peu de données et mon petit PC va y arriver sans qu'il soit besoin de simplifier le problème. **Cela fait donc 17 colonnes à organiser et une matrice 5380*17 à créer.** 


Si certaines colonnes sont déjà dans un format numérique pertinent, **d'autres doivent être reformatées intelligemment** car il s'agit de string. Comme j'ai en tête d'utiliser des algorithmes assez simples (et pas un NN qui pourrait faire le job), il faut autant que possible transformer les chaînes de caractères en nombres qui sont pertinents pour ces algorithmes. Typiquement, pour les niveaux d'éducation, il semble pertinent de poser par exemple : 0 pour 'Secondary or lower', 1 pour University -> 1, 2 pour Diploma et 3 pour Master/phD (même si je ne suis pas sûr de comprendre ce que Diploma veut dire pour la banque)...


NB : je ne maîtrise pas très bien toutes les subtilités de Python. Plutôt que de passer des heures à trouver la parfaite formulation en Pyton, j'ai gagné du temps en modifiant le fichier initial (sous Excel) pour le réenregistrer avec des ; comme séparateurs et des . pour indiquer les parties décimales (sinon, mon csv.reader confondait le séparateur de champ et celui indiquant la partie décimale). J'ai aussi approximé la date par 365*(année-1900)+30*mois+jour.

In [2]:
# Read and organise the data
index=0
nbdata=5381
X=np.zeros((nbdata,17))
Y=np.zeros(nbdata)

with open('CreditTraining2.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quoting=csv.QUOTE_NONE)
    next(spamreader)
   
    for row in spamreader:
        Y[index]=row[1]

        if (row[2]=='Non Existing Client'):
            X[index,0]=1 # Sinon on est déjà à 0
            
        date = datetime.strptime(row[3], '%d/%m/%Y').date() #Bith date
        X[index,1]=date.day+date.month*30+(date.year-1900)*365 # Approximation qui suffira 
        
        date= datetime.strptime(row[4], '%d/%m/%Y').date() # Customer_Open_Date
        X[index,2]=date.day+date.month*30+(date.year-1900)*365 
        
        if (row[5]=='NP_Client'):
            X[index,3]=1 #Sinon on est déjà à 0

        if (row[6]=='University'): # Cas Secondary or less reste à 0
            X[index,4]=1 
        if (row[6]=='Diploma'):
            X[index,4]=2 
        if (row[6]=='Master/PhD'):
            X[index,4]=3 
            
        if (row[7]=='Separated'):   # Cas Divorced reste à 0. Intuitivement c'est le pire pour la banque 
            X[index,5]=1           # Il peut y avoir une pension alimentaire...
        if (row[7]=='Widowed'):
            X[index,5]=2 
        if (row[7]=='Single'):
            X[index,5]=3 
        if (row[7]=='Married'):     # Meilleure situation à mon sens - En tous cas la plus éloignée de Divorced
            X[index,5]=4    
        
        if (row[8]!='') :
            X[index,6]=row[8] # Number_Of_Dependant - Attention deux lignes vides qui resteront donc à zéro
            
        X[index,7]=row[9] # Years_At_Residence
        
        if (row[10]!=''):
            X[index,8]=float(row[10].replace(',','.')) # Net_Annual_Income - Attention deux lignes vides qui resteront donc à zéro
        
        if (row[11]!=''):
            X[index,9]=row[11] # Years_At_Business - Attention deux lignes vides qui resteront donc à zéro
        
        if (row[12]=='P'):   # Le C reste à 0
            X[index,10]=1
        if (row[12]=='G'):    
            X[index,10]=2   
            
        date= datetime.strptime(row[13], '%d/%m/%Y').date() # Years_At_Business
        X[index,11]=date.day+date.month*30+(date.year-1900)*365 
        
        if (row[14]=='Sales'):   # Le Branch reste à 0
            X[index,12]=1
        
        if (row[15]=='New rent'):
            X[index,13]=1
        if (row[15]=='Old rent'):
            X[index,13]=2 
        if (row[15]=='Company'):
            X[index,13]=3 
        if (row[15]=='Owned'):     
            X[index,13]=4           
        
        X[index,14]=row[16] # Nb_Of_Products

        if (row[17]!=''):     # Prod_Closed_Date - une majorité de lignes vides
            date= datetime.strptime(row[17], '%d/%m/%Y').date() # Years_At_Business
            X[index,15]=date.day+date.month*30+(date.year-1900)*365 
        else :
            X[index,15]=114*365+1 # 1er janvier 2014 soit après toutes les autres dates  
         
        X[index,16]=ord(row[18])    
        
        index=index+1        

Bien sûr, j'ai fait quelques print pour vérifier que les datas étaient correctement chargées. C'est le cas.

## 3. Posons nous un peu sur ce problème !

Le problème proposé est de modéliser pour savoir s'il faut accorder ou non un prêt et plus précisément si l'emprunteur va faire défaut ou pas. Il s'agit d'un problème de classification binaire mais **avec une très forte proportion de Y=0 (92.7%)** représentant les clients qui ne font pas défaut et remboursent bien leur prêt. Il y a à ce stade **quatre remarques importantes** :

- Nous sommes dans une situation où **il faut éviter d'attribuer un prêt à une personne qui ferait défaut** ; c'est beaucoup plus grave que d'en refuser un à une personne qui, en fait, rembourserait. Autrement dit, le problème, c'est le cas Ypred=0 alors que Yreal=1 soit les faux négatif (FN). Concrètement, si l'algorithme est utilisé en production, la banque va refuser de prêter si Ypred=1 ou on aura un taux de défaut de FN/(FN+TN). Les FP sont moins grave mais ils posent aussi problème en privant la banque d'une source de revenu. Une métrique pertinente de la qualité de l'algorithme pourrait donc pas exemple de **minimiser (FP+3*FN)/(FP+FN+TP+TN)**. Comme mon facteur 3 est purement empirique et que cette métrique n'est pas commune, **je compare aussi le score  F1=2xPrecisionxRecall/(Precision+Recall)=2TP/(2TP+FN+FP)**. J'affiche enfin le taux de chiffre d'affaire perdu "inutilement" et l'amélioration du taux de défaut par rapport à la situation où l'algorithme ne serait pas utilisé.


- **Si** (comme cela semble très probable) **les données réelles ont été produites par des agents de la banque, elles sont assez imparfaites** et il sera difficile d'atteindre de très bons scores à partir de ces données.


- **L'échantillon est fort réduit surtout sur les Y=1 (seulement 393 cas)**. On va donc avoir des effet de fluctuation statisitique.


- Cela sort du cadre de l'exercice mais **nous travaillons sur des données du passé** (et forcément un passé un peu ancien pour savoir si le client a fait ou non défaut). On doit donc supposer que le comportement des clients dans le futur est le même que dans le passé... et qu'aucune grosse crise économique ne va complètement changer les choses !

Je procède en 3 étapes pour la suite :

- **Je coupe le dataset en un échantillon de training et un autre de test.** Je garde 35% pour la validation finale. Avec 20% et 25%, mes résultats étaient moins bons (overfitting ?).


- **Je fais de l'oversampling de la classe Y=1 pour le training set.** J'ai également essayé de faire fonctionner Random_Forest sans ce resampling mais en ajustant les poids comme suggéré en cours (class_weight='balanced' en paramètre) : les résultats sont meilleurs avec ces poids que sans mais moins bon qu'en faisant de l'oversampling. J'ai testé trois algorithmes de resampling et gardé celui qui me donnait les meilleurs résultats.


- **Je Teste différents classifier et sélectionne le meilleur en comparant les scores de validation** La Random Forest donne des résultats assez nettement meilleurs que les autres et il n'y a pas d'hyperparamètre à régler (on peut juste jouer sur le nombre d'arbre ; plus il y a d'arbres et meilleure est la prévision ... aux fluctuations aléatoires prêt). **Je ne coupe donc pas mon training set en deux pour avoir un validation set** et utilise directement le testing set pour comparer les modèles.

## 4. Fabriquer les ensemble

In [3]:
# Split into Training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42,shuffle=True)

In [4]:
#Check that there is roughly the same proportion of Y=1 in both samples
print(y_train.mean())
print(y_test.mean())

0.07091792965398913
0.07696390658174097


In [5]:
# Ressample Trainig set - I tried 3 algorithms and kept the sampliest one
X_resampled, y_resampled = RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
#X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
#X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

# and check
print(X_resampled.shape)
print(y_resampled.shape)
print(y_resampled.mean())

(6498, 17)
(6498,)
0.5


## 5. Essai de différents classifiers et comparaison des résultats

In [6]:
#Petite fonction pour indiquer les résultats
def get_result(y):
    print("Confusion matrix :")
    print(confusion_matrix(y_test, y))
    FP=confusion_matrix(y_test, y)[0][1]
    FN=confusion_matrix(y_test, y)[1][0]
    TP=confusion_matrix(y_test, y)[1][1]
    TN=confusion_matrix(y_test, y)[0][0]
    print("Taux de chiffre d'affaire perdu sans raison :",FP/(TN+FP))
    print("Le taux de défaut devrait être :",FN/(TN+FN)," - Avant il était de ",(FN+TP)/((FP+FN+TP+TN)))
    print("Score maison à minimiser :",(FP+3*FN)/(FP+FN+TP+TN))
    print("score F1 : ",f1_score(y_test, y))
    print("")

In [7]:
# KNN - Je ne mets pas tous les résultats, ils sont décevants
for n_neighbors in [3,9,51]:
    print("ESSAI avec n_neighbors=",n_neighbors)
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_resampled, y_resampled) 
    ypred_nn=knn.predict(X_test)
    get_result(ypred_nn)

ESSAI avec n_neighbors= 3
Confusion matrix :
[[1594  145]
 [  62   83]]
Taux de chiffre d'affaire perdu sans raison : 0.08338125359401956
Le taux de défaut devrait être : 0.03743961352657005  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.1756900212314225
score F1 :  0.4450402144772118

ESSAI avec n_neighbors= 9
Confusion matrix :
[[1493  246]
 [  26  119]]
Taux de chiffre d'affaire perdu sans raison : 0.14146060954571593
Le taux de défaut devrait être : 0.017116524028966424  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.17197452229299362
score F1 :  0.4666666666666667

ESSAI avec n_neighbors= 51
Confusion matrix :
[[1507  232]
 [  22  123]]
Taux de chiffre d'affaire perdu sans raison : 0.13341000575043127
Le taux de défaut devrait être : 0.014388489208633094  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.15817409766454352
score F1 :  0.4919999999999999



In [9]:
# Essai de AdaBoost
for n_estimators in [7,23,45,101,303,1000]:
    print ("ESSAI AVEC n_estimators=",n_estimators)
    clf = AdaBoostClassifier(n_estimators=n_estimators, random_state=42)
    clf.fit(X_resampled, y_resampled)
    ypred_ada = clf.predict(X_test)
    get_result(ypred_ada)

ESSAI AVEC n_estimators= 7
Confusion matrix :
[[1565  174]
 [  10  135]]
Taux de chiffre d'affaire perdu sans raison : 0.10005750431282347
Le taux de défaut devrait être : 0.006349206349206349  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.10828025477707007
score F1 :  0.5947136563876653

ESSAI AVEC n_estimators= 23
Confusion matrix :
[[1574  165]
 [  11  134]]
Taux de chiffre d'affaire perdu sans raison : 0.09488211615871191
Le taux de défaut devrait être : 0.00694006309148265  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.10509554140127389
score F1 :  0.6036036036036035

ESSAI AVEC n_estimators= 45
Confusion matrix :
[[1583  156]
 [  14  131]]
Taux de chiffre d'affaire perdu sans raison : 0.08970672800460035
Le taux de défaut devrait être : 0.008766437069505322  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.10509554140127389
score F1 :  0.6064814814814815

ESSAI AVEC n_estimators= 101
Confusion matrix :
[[1606  1

In [10]:
# Try Random Forest without using resampling 
for random_state in range (43,46):
    print ("ESSAI AVEC Random_state=",random_state)
    rf = RandomForestClassifier(n_estimators = 100, random_state = random_state,class_weight='balanced')
    rf.fit(X_train, y_train)
    ypred_rfnr = rf.predict(X_test)
    get_result(ypred_rfnr)
    
# A OBSERVER : LA FORTE VARIABILITE EN FONCTION DU Random_state !

ESSAI AVEC Random_state= 43
Confusion matrix :
[[1728   11]
 [  75   70]]
Taux de chiffre d'affaire perdu sans raison : 0.0063254744105807935
Le taux de défaut devrait être : 0.04159733777038269  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.12526539278131635
score F1 :  0.6194690265486726

ESSAI AVEC Random_state= 44
Confusion matrix :
[[1724   15]
 [  71   74]]
Taux de chiffre d'affaire perdu sans raison : 0.008625646923519264
Le taux de défaut devrait être : 0.03955431754874652  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.12101910828025478
score F1 :  0.6324786324786326

ESSAI AVEC Random_state= 45
Confusion matrix :
[[1726   13]
 [  70   75]]
Taux de chiffre d'affaire perdu sans raison : 0.007475560667050029
Le taux de défaut devrait être : 0.03897550111358575  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.1183651804670913
score F1 :  0.6437768240343348



In [11]:
# Try Random Forest with resampling : better
for random_state in range (43,46):
    print ("ESSAI AVEC Random_state=",random_state)
    rf = RandomForestClassifier(n_estimators = 100, random_state = random_state)
    rf.fit(X_resampled, y_resampled)
    ypred_rf = rf.predict(X_test)
    get_result(ypred_rf)
    
# IDEM : LA FORTE VARIABILITE EN FONCTION DU Random_state !

ESSAI AVEC Random_state= 43
Confusion matrix :
[[1709   30]
 [  46   99]]
Taux de chiffre d'affaire perdu sans raison : 0.017251293847038527
Le taux de défaut devrait être : 0.02621082621082621  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.08917197452229299
score F1 :  0.7226277372262774

ESSAI AVEC Random_state= 44
Confusion matrix :
[[1707   32]
 [  45  100]]
Taux de chiffre d'affaire perdu sans raison : 0.018401380103507763
Le taux de défaut devrait être : 0.025684931506849314  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.08864118895966029
score F1 :  0.7220216606498194

ESSAI AVEC Random_state= 45
Confusion matrix :
[[1708   31]
 [  43  102]]
Taux de chiffre d'affaire perdu sans raison : 0.017826336975273145
Le taux de défaut devrait être : 0.024557395773843516  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.08492569002123142
score F1 :  0.7338129496402878



In [12]:
# Le même en faisant varier le nombre d'arbres
random_state=45
for n_trees in [11,31,100,201,301]:
    print ("ESSAI AVEC n_trees=",n_trees)
    rf = RandomForestClassifier(n_estimators = n_trees, random_state = random_state)
    rf.fit(X_resampled, y_resampled)
    ypred_rf = rf.predict(X_test)
    get_result(ypred_rf)

ESSAI AVEC n_trees= 11
Confusion matrix :
[[1708   31]
 [  46   99]]
Taux de chiffre d'affaire perdu sans raison : 0.017826336975273145
Le taux de défaut devrait être : 0.026225769669327253  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.08970276008492568
score F1 :  0.7200000000000001

ESSAI AVEC n_trees= 31
Confusion matrix :
[[1710   29]
 [  44  101]]
Taux de chiffre d'affaire perdu sans raison : 0.01667625071880391
Le taux de défaut devrait être : 0.02508551881413911  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.08545647558386411
score F1 :  0.7345454545454545

ESSAI AVEC n_trees= 100
Confusion matrix :
[[1708   31]
 [  43  102]]
Taux de chiffre d'affaire perdu sans raison : 0.017826336975273145
Le taux de défaut devrait être : 0.024557395773843516  - Avant il était de  0.07696390658174097
Score maison à minimiser : 0.08492569002123142
score F1 :  0.7338129496402878

ESSAI AVEC n_trees= 201
Confusion matrix :
[[1708   31]
 [  42  103]

## 6. Conclusion

En gardant 35% de l'échantillon en testing set pour la validation, et en entrainement les 65% après les avoir over samplés (avec RandomOverSampler), j'obtiens mes meilleurs résultats avec **une Foret de 301 arbres : 74% de score F1**. Il faut noter qu'il y a une variation importante due au choix du random_state. **Ce représente une division par plus de 3 du taux de défaut !**

Ce niveau autour de 74% n'est pas très élevé mais les données fournies sont assez peu importantes en quantité et, surtout, elles doivent contenir elle-même une part d'erreur : deux agents différents de la même banque, avec les mêmes données, ont probablement des Y qui peuvent être différents (et 74% de F1 correspond tout de même à 96.23% d'accuracy).

**IMPORTANT : si le meilleur score F1 est obtenu s'assez loin avec RamdomForest, c'est BEAUCOUP moins clair pour mon 'score maison' qui cherche à minimiser 3*FN + FP. Si on prenait 10*FN+FP, Adaboost serait meilleur car il donne nettement moins de FN (10 à 20 contre 40 à 50 pour RF). D'ailleurs, certains résultats d'Adaboost montrent une division par 10 du taux de défaut (mais avec aussi un fort taux de rejet de dossier qu'il aurait fallu accepter). Pour aller plus loin et vraiment conclure, il faudrait connaître le coût économique d'un défaut du client et le comparer au coût économique d'un prêt non accordé (alors qu'il aurait été remboursé), c'est-à-dire choisir la fonction de perte en fonction de paramètres "réels" ; c'est sans doute au delà de cet exercice.**

A noter : pour finir, une fois le modèle sélectionné, il faudrait le refaire tourner sur 100% du dataset ce qui devrait un peu améliorer la performance. Mais bien sûr, faute de testing set, on ne pourrait alors qu'estimer sa performance par une cross validation